In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from datetime import datetime
import sqlite3


# Set up the web driver (make sure to install the appropriate WebDriver, like ChromeDriver)
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Run headlessly for better performance, but no window

options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Users\wilman\py_envs\yahoo_dfs\data",  # Set your download folder path here
    "download.prompt_for_download": False,
    "download.directory_upgrade": True, 
    #"safebrowsing.enabled": True
})


driver = webdriver.Chrome(service=webdriver.chrome.service.Service('../src/chromedriver-win32/chromedriver.exe'), options=options)


Method #1: Gather lineup for the DFS

In [33]:
contest_id = ""
driver.get(f"https://sports.yahoo.com/dailyfantasy/contest/{contest_id}/setlineup")

# click out of popups
## TBD

# Scroll down to load content until there’s no more new content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait for content to load
    time.sleep(3)
    
    # Check if more content has been loaded
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [51]:
# Locate and extract player names and salaries using XPath
players_data = []
today = datetime.today().date()
base_url = "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@id='content']/div[@class='Mx(a) Pb(10px) Bbbw(1px) Bdbs(s) Bdbc($c-fuji-dirty-seagull) W(1080px) Thm-lite']/div[@class='Pos(r)']/div[@class='IbBox W(3/5)']/div[@id='jumptocontent']/div[@id='primary-1-DesktopLineupSelection-Proxy']/div[@id='lineupSelection']/div[@class='Px(10px)']/div[@class='Pb(10px) Px(10px)']/div[@class='Us(n) Mih(670px)']/table[@class='W(100%) Hover TableRowBorderTop TableVa(m) Mt(-1px) LinkColorOnHover']/tbody[@class='infinite-scroll-list']/tr[@class='Cur(p)']"
n_players = 200
for player in range(1,n_players+1):
    player_name_xpath = f"{base_url}[{player}]/td[@class='Px(6px) Py(10px)']/a[@class='Td(n)']"
    salary_xpath = f"{base_url}[{player}]/td[@class='Px(6px) Py(10px) Ta(end)'][3]/span/span"
    pos_xpath = f"{base_url}[{player}]/td[@class='Px(6px) Py(10px) Ta(end)'][1]/span[@class='Fw(b)']"
    FPPG_xpath = f"{base_url}[{player}]/td[@class='Px(6px) Py(10px) Ta(end) Whs(nw)']/div/span[@class='IbBox Va(m) Pstart(4px)']"
    std_xpath = 
    
    try:
        pos = driver.find_element(By.XPATH, pos_xpath).text  
        name = driver.find_element(By.XPATH, player_name_xpath).text 
        salary = driver.find_element(By.XPATH, salary_xpath).text  
        FPPG = driver.find_element(By.XPATH, FPPG_xpath).text  
        players_data.append({"Pos":pos, "Name": name, "FPPG": FPPG,"Salary": salary, "Date":today})
    except Exception as e:
        print("Error extracting player data:", e)

In [52]:
df = pd.DataFrame(players_data)
df.to_csv(f"../data/available_players_{str(today)}.csv")

In [53]:
df.head()

,Pos,Name,FPPG,Salary,Date
0,PF,Giannis Antetokounmpo,52.3,$55,2024-10-25
1,C,Joel Embiid,60.5,$54,2024-10-25
2,C,Anthony Davis,72.2,$54,2024-10-25
3,PF,Kevin Durant,38.4,$49,2024-10-25
4,SF,LeBron James,32.0,$48,2024-10-25


Method #2: Download available players from UI

In [34]:
contest_id = "14255365"
driver.get(f"https://sports.yahoo.com/dailyfantasy/contest/{contest_id}/setlineup")

In [35]:
info_icon = wait.until(EC.element_to_be_clickable((By.XPATH, "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@id='content']/div[@class='Mx(a) Pb(10px) Bbbw(1px) Bdbs(s) Bdbc($c-fuji-dirty-seagull) W(1080px) Thm-lite']/div[2]/div[@id='top-0-ContestDetailsHeaderWrapper-Proxy']/div[@class='P(10px) Pb(0)']/div/div[@class='Py(10px) Pb(0) Mb(10px) Cf Bb-accent']/div[@class='IbBox Pos(r)']/div[@class='D(ib) Va(m)']/div[@class='IbBox']/h4[@class='Fw(400) Fz(fz-xl) Mb(4px) Mt(0)']/span[@class='Fw(200)']/i[@class='ys-icon-info_outline C(button-primary-color) Cur(p) M(6px) Fz(15px)']")))
info_icon.click()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00573853+24035]
	(No symbol) [0x004FBBE4]
	(No symbol) [0x003DC2D3]
	(No symbol) [0x003BE00B]
	(No symbol) [0x00445F4F]
	(No symbol) [0x00458E99]
	(No symbol) [0x0043FC26]
	(No symbol) [0x0041218C]
	(No symbol) [0x0041310D]
	GetHandleVerifier [0x00819683+2800659]
	GetHandleVerifier [0x0087423E+3172302]
	GetHandleVerifier [0x0086CE52+3142626]
	GetHandleVerifier [0x00616C00+692624]
	(No symbol) [0x00504BFD]
	(No symbol) [0x00501908]
	(No symbol) [0x00501AA0]
	(No symbol) [0x004F3F50]
	BaseThreadInitThunk [0x75997BA9+25]
	RtlInitializeExceptionChain [0x77A6C0CB+107]
	RtlClearBits [0x77A6C04F+191]


In [29]:
contest_id = "14255365"
driver.get(f"https://sports.yahoo.com/dailyfantasy/contest/{contest_id}/setlineup")

info_icon = wait.until(EC.element_to_be_clickable((By.XPATH, "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@id='content']/div[@class='Mx(a) Pb(10px) Bbbw(1px) Bdbs(s) Bdbc($c-fuji-dirty-seagull) W(1080px) Thm-lite']/div[2]/div[@id='top-0-ContestDetailsHeaderWrapper-Proxy']/div[@class='P(10px) Pb(0)']/div/div[@class='Py(10px) Pb(0) Mb(10px) Cf Bb-accent']/div[@class='IbBox Pos(r)']/div[@class='D(ib) Va(m)']/div[@class='IbBox']/h4[@class='Fw(400) Fz(fz-xl) Mb(4px) Mt(0)']/span[@class='Fw(200)']/i[@class='ys-icon-info_outline C(button-primary-color) Cur(p) M(6px) Fz(15px)']")))
info_icon.click()

time.sleep(5)

# downloads the lineup for the contest

export_player_list = wait.until(EC.element_to_be_clickable((By.XPATH, "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@class='Z(11) End(0) Pos(f) Start(0) T(0)']/div/div[@id='overlay-1-Lightbox-Proxy']/div[@id='overlay-1-Lightbox']/div[@class='lightbox-wrapper Ta(c) Pos(f) T(0) Start(0) H(100%) W(100%) PageOverlay Z(50) Op(1)']/div[@id='myLightboxContainer']/div[@class='Pos(r)']/div[@class='ys-contestModal ys-modalContent Px(20px) Thm-lite']/div[@class='Mih(400px)']/div[@class='Fxg(1)']/div[2]/div[@class='Bt-lite bd Ov(a) Pt(10px) Mih(400px) Pos(r)']/div[@class='bd']/div[@class='Thm-lite Fz(fz-sm)']/div[@class='IbBox W(1/1)']/div[@class='Py(10px)']/p[@class='IbBox M(0) Fz(fz-xsm) Fl(end)']/a")))
export_player_list.click()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00573853+24035]
	(No symbol) [0x004FBBE4]
	(No symbol) [0x003DC2D3]
	(No symbol) [0x003BE00B]
	(No symbol) [0x00445F4F]
	(No symbol) [0x00458E99]
	(No symbol) [0x0043FC26]
	(No symbol) [0x0041218C]
	(No symbol) [0x0041310D]
	GetHandleVerifier [0x00819683+2800659]
	GetHandleVerifier [0x0087423E+3172302]
	GetHandleVerifier [0x0086CE52+3142626]
	GetHandleVerifier [0x00616C00+692624]
	(No symbol) [0x00504BFD]
	(No symbol) [0x00501908]
	(No symbol) [0x00501AA0]
	(No symbol) [0x004F3F50]
	BaseThreadInitThunk [0x75997BA9+25]
	RtlInitializeExceptionChain [0x77A6C0CB+107]
	RtlClearBits [0x77A6C04F+191]


## Download Contest History

Note: yahoo only does last 6 months from date

In [ ]:
## download contests

# Open the website
driver.get(f"https://sports.yahoo.com/dailyfantasy/contests/history")

# Wait for the element to be clickable and click it
wait = WebDriverWait(driver, 20)
download_context_history_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@id='content']/div[@class='Mx(a) Pb(10px) Bdbw(1px) Bdbs(s) Bdbc($c-fuji-dirty-seagull) W(1080px) Thm-lite']/div[@id='jumptocontent']/div[@id='main-0-DesktopContestsHistory-Proxy']/div[@class='Thm-lite P(10px)']/div[@class='Pt(10px)']/div[@class='D(f) Jc(sb) Bb-accent Mb(20px)']/div[@class='IbBox']/button[@class='D(if) Jc(c) Ai(c) Bdrs(9999px) Cur(p) O(button-focus-visible):fv Cur(d):di Op(1):di C(#b0b9c1)!:di Pe(n):di Mb(6px) Bgc(t) C($c-scooter)! Bd(light-grey-border) Bgc(bgc-trans-light-grey):h Bdc(#b0b9c1):di Px(16px) Py(4px) Fw(500) Fz(16px) Lh(1.5)']")))
download_context_history_button.click()

# Wait for the download to complete if needed (use your logic)
time.sleep(5)

download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@class='Z(11) End(0) Pos(f) Start(0) T(0)']/div/div[@id='overlay-1-Lightbox-Proxy']/div[@id='overlay-1-Lightbox']/div[@class='lightbox-wrapper Ta(c) Pos(f) T(0) Start(0) H(100%) W(100%) PageOverlay Z(50) Op(1)']/div[@id='myLightboxContainer']/div[@class='ys-download-contest-history-modal P(16px) Bgc(white) Pos(r) Bdrs(8px) Miw(392px) Maw(392px) smartphone_W(100%) smartphone_Bxz(bb) smartphone_Bdrststart(8px) smartphone_Bdrsbend(0) smartphone_Bdrsbstart(0) smartphone_Bdrstend(8px) smartphone_Miw(ini) smartphone_Maw(ini)']/a[@class='YahooSans-Medium D(f) Mb(10px) Fz(16px) W(100%) H(52px) Bdrs(26px) Bgc($c-fuji-blue-1-a) C(white) Mt(16px) Ai(c) Jc(c) Td(n):h Bxz(bb)']")))
download_button.click()
time.sleep(10)



## Gather Entries Data from Contests

In [12]:
conn = sqlite3.connect('../data/fantasy.db')
cursor = conn.cursor()


In [15]:
# Gather contests (could filter more by start_date)
cursor.execute("Select id, entry_id from contests;")

contests_ls = cursor.fetchall()

In [19]:

base_contest_url = "/html[@id='atomic']/body/div[@id='app']/div[@id='page']/div/div[@id='render-target-default']/div/div[@id='content']/div[@class='Mx(a) Pb(10px) Bdbw(1px) Bdbs(s) Bdbc($c-fuji-dirty-seagull) W(1080px) Thm-lite']/div[@id='jumptocontent']/div[@id='main-0-ContestDetails-Proxy']/div[@class='Pos(r) P(10px)']/div[@class='D(f)']/div[@class='IbBox W(45%) Bxz(bb) D(b)']/div/table[@class='W(100%) TableVa(m) M(0) Bd(0) LinkColorOnHover']/tbody/tr[@class='Bb-lite Bxz(bb) H(61px) Cur(p)']"
for contest_id, entry_id in contests_ls:

    # open contest
    driver.get(f"https://sports.yahoo.com/dailyfantasy/contest/{contest_id}/{entry_id}")
    time.sleep(5)

    # 8 players in a roster
    for i in range(1,9):
        player_name_xpath = f"{base_contest_url}[{i}]/td[@class='Py(0) Pos(r) Ta(end)']/div[@class='Va(m) IbBox Maw(69%)']/div/div[1]/a[@class='C(font-color-default)!']"
        FPPG_xpath = f"{base_contest_url}[{i}]/td[@class='Ta(c)'][1]"
        points_xpath = f"{base_contest_url}[{i}]/td[@class='Ta(c) Fw(b)']/div[@class='D(ib) Mb(2px)']/span[@class='ydfs-scoring-points']"
        

        try: 
            player_name = driver.find_element(By.XPATH, player_name_xpath).text
            FPPG_name = driver.find_element(By.XPATH, player_name_xpath).text
            




In [ ]:
cursor.execute("Select id, entry_id from contests;")
cursor.fetchall()

In [ ]:
# Quit the driver
driver.quit()
